In [16]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

In [17]:
df = pd.read_csv('../Data/chatbot_training_data.csv')

In [18]:
x = df['text'].values
y = df['intent'].values


In [19]:
params = {
    'svc__C': [1, 5, 10],
    'svc__kernel': ['linear', 'rbf'],
    'svc__class_weight': [None, 'balanced'],
    'svc__gamma': ['scale', 'auto']
}

In [20]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), stop_words='english')),
    ('svc', SVC(probability=True))
])

In [ ]:

grid = GridSearchCV(pipeline, param_grid=params, cv=5)
grid.fit(x, y)

In [22]:
print("Best Params:", grid.best_params_)
print("Best Cross-Validated Accuracy:", grid.best_score_)


Best Params: {'svc__C': 1, 'svc__class_weight': 'balanced', 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best Cross-Validated Accuracy: 0.8002016129032258


In [23]:
y_pred = grid.predict(x)  
print("Accuracy:", accuracy_score(y, y_pred))
print(classification_report(y, y_pred))

Accuracy: 0.9871794871794872
              precision    recall  f1-score   support

   faq_hours       1.00      1.00      1.00        20
 faq_payment       1.00      1.00      1.00        20
 faq_returns       1.00      1.00      1.00        20
faq_shipping       1.00      1.00      1.00        20
       greet       1.00      0.95      0.97        20
  human_help       1.00      0.95      0.97        20
          ok       0.89      1.00      0.94        16
order_status       1.00      1.00      1.00        20

    accuracy                           0.99       156
   macro avg       0.99      0.99      0.99       156
weighted avg       0.99      0.99      0.99       156



In [24]:
ypred = grid.predict(['update on my order'])
print(ypred)

['order_status']


In [25]:

def clean_input(text):
    text = text.lower()
    text = re.sub(r'[^\w\s#]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [28]:

def predict_intent(user_input):
    prediction = grid.predict([user_input])[0]
    probability = grid.predict_proba([user_input])[0]
    confidence = np.max(probability)
    return prediction, float(confidence)


In [ ]:
def get_response(intent, user_input):
    if intent == 'greet':
        return "Hello there! How can I help you today?"
    elif intent == 'ok':
        return "Okay! If you have any questions, feel free to ask or type 'quit' to exit."
    elif intent == "order_status":
        order_id = re.findall(r'#?\d{4,6}', user_input)
        order_id = order_id[0].lstrip('#') if order_id else None
        mock_orders = {
            "12345": "📦 Order 12345 is out for delivery.",
            "67890": "✅ Order 67890 has been delivered."
        }
        return mock_orders.get(order_id, "⚠️ I couldn't find that order ID.") if order_id else "Please provide your order number."
    elif intent == "faq_payment":
        return "💳 We accept Visa, MasterCard, and PayPal."
    
    elif intent == "faq_hours":
        return "🕘 Our store hours are 9 AM to 9 PM, Monday to Saturday."

    elif intent == "faq_returns":
        return "🔄 You can return items within 30 days of purchase with a receipt."

    elif intent == "human_help":
        return "☎️ Please hold while I connect you to a human representative."

    elif intent == "faq_shipping":
        return "🚚 We offer free shipping on orders over $50. Standard shipping takes 3-5 business days."

    elif intent == "faq_contact":
        return "📞 You can reach us at ABC@company.com or 1-800-555-01**"
    else:
        return "❓ I'm not sure how to help with that. Please rephrase or contact support at 1-800-555-01**."

def chatbot_response():
    print("🤖 Bot: Hello! How can I assist you today?")
    while True:
        user_input = input("🧑 You: ").strip()
        if user_input.lower() in ["bye", "exit", "quit"]:
            print("🤖 Bot: Goodbye! Have a great day!")
            break
        cleaned_input = clean_input(user_input)
        intent, confidence = predict_intent(cleaned_input)
        if confidence < 0.4:
            print(f"🤖 Bot: Hmm... I'm not confident in my understanding (Intent: {intent}, Confidence: {confidence:.2f}). Could you rephrase?")
        else:
            response = get_response(intent, user_input)
            print("🕵️‍♂️ you: ",user_input)
            print(f"🤖 Bot (Intent: {intent}, Confidence: {confidence:.2f}): {response}")
chatbot_response()


🤖 Bot: Hello! How can I assist you today?
🤖 Bot: Goodbye! Have a great day!
